In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install lightgbm==3.3.5

In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [7]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [8]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [9]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSEmploymentDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_preprocessing_intervention

## Define Input Variables

In [10]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0 + 0.1 * i for i in range(11)]
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5,
#                                 0.6000000000000001, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = [1.0]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'folk_ny_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [11]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [12]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [13]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5


## Initialize custom objects

In [14]:
data_loader = ACSEmploymentDataset(state=['NY'], year=2018, with_nulls=False,
                                   subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,5,4,0,1,2,1,2,2,2,1,1,2,1,1,15,42
1,1,4,0,1,2,1,2,2,2,2,1,0,1,2,21,59
2,2,4,0,1,2,2,2,2,2,2,1,0,4,4,19,78
3,5,4,0,1,2,1,2,2,2,1,1,10,1,4,14,19
4,1,2,0,1,2,1,2,2,2,1,1,0,1,1,17,87


In [15]:
data_loader.X_data.shape

(20000, 16)

## Run experiment iterations

### Experiment iteration 1

In [32]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [33]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [34]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             with_tuning=True,
#                                              with_tuning=False,
#                                              tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-06 13:27:22 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[1.0]',
 'model_init_seed': 100,
 'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5'}




2023-07-06 13:27:23 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

...

2023-07-06 13:55:11 experiment_interface.py INFO    : Models are tuned and saved to a file


2023/07/06, 13:55:11: Tuning for RandomForestClassifier is finished [F1 score = 0.7770552282956381, Accuracy = 0.7868333333333334]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [45]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-06 15:12:44 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, '
                                 '0.5, 0.6000000000000001, 0.7, 0.8, 0.9, 1.0]',
 'model_init_seed': 200,
 'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5'}




2023-07-06 15:12:44 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.7
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels/tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 7

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 7
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 8

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 8
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

### Experiment iteration 9

In [16]:
# Configs for an experiment iteration
exp_iter_num = 9
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [17]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-06 18:37:26 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 900,
 'experiment_iteration': 'Exp_iter_9',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, '
                                 '0.5, 0.6000000000000001, 0.7, 0.8, 0.9, 1.0]',
 'model_init_seed': 900,
 'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5'}




2023-07-06 18:37:26 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([11778,  7394,  3707,   179,   582,  8895, 17131,  7128,  5974,
             8797, 19077,  5615,  7347, 14549,  7747,  9891, 15233, 10558,
            11493,  7993],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([11778,  7394,  3707,   179,   582,  8895, 17131,  7128,  5974,
             8797, 19077,  5615,  7347, 14549,  7747,  9891, 15233, 10558,
            11493,  7993],
           dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  1.0
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels/tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': False, 'random_state': 901, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 10

In [16]:
# Configs for an experiment iteration
exp_iter_num = 10
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.0_20230705__174859.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.1_20230705__182108.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.2_20230705__185830.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.30000000000000004_20230705__194739.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.4_20230705__202544.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.5_20230705__210307.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.6000000000000001_20230705__214028.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.7_20230706__095236.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv',
#     'tuning_results_Folktables_NY_2018_Employment_alpha_0.9_20230706__161158.csv',
    'tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [17]:
run_exp_iter_with_preprocessing_intervention(data_loader=exp_iter_data_loader,
                                             experiment_seed=experiment_seed,
                                             test_set_fraction=TEST_SET_FRACTION,
                                             db_writer_func=db_writer_func,
                                             fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                             column_transformer=preprocessor,
                                             models_params_for_tuning=models_params_for_tuning,
                                             metrics_computation_config=metrics_computation_config,
                                             custom_table_fields_dct=custom_table_fields_dct,
                                             # with_tuning=True,
                                             with_tuning=False,
                                             tuned_params_df_paths=tuned_params_df_paths,
                                             save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                             verbose=True)

2023-07-07 09:44:35 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 1000,
 'experiment_iteration': 'Exp_iter_10',
 'fair_intervention_params_lst': '[1.0]',
 'model_init_seed': 1000,
 'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5'}




2023-07-07 09:44:35 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 7082,  7324,  3040,  5913,  1696, 16043,  8692, 18603, 15738,
             6766, 18547, 15134, 15569,  6841,  1627,  4267,   832, 18568,
             2397, 13521],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 7082,  7324,  3040,  5913,  1696, 16043,  8692, 18603, 15738,
             6766, 18547, 15134, 15569,  6841,  1627,  4267,   832, 18568,
             2397, 13521],
           dtype='int64')


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2023-07-07 09:44:35 experiment_interface.py INFO    : Models config is loaded from the input file


intervention_param:  1.0
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels/tuning_results_Folktables_NY_2018_Employment_alpha_1.0_20230706__175511.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 50, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': False, 'random_state': 1000, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]